# Distirbuted Training of Mask-RCNN in Amazon SageMaker using S3

This notebook is a step-by-step tutorial on distributed tranining of [Mask R-CNN](https://arxiv.org/abs/1703.06870) implemented in [TensorFlow](https://www.tensorflow.org/) framework. Mask R-CNN is also referred to as heavy weight object detection model and it is part of [MLPerf](https://www.mlperf.org/training-results-0-6/).

Concretely, we will describe the steps for training [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) and [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) in [Amazon SageMaker](https://aws.amazon.com/sagemaker/) using [Amazon S3](https://aws.amazon.com/s3/) as data source.

The outline of steps is as follows:

1. Stage COCO 2017 dataset in [Amazon S3](https://aws.amazon.com/s3/)
2. Build SageMaker training image and push it to [Amazon ECR](https://aws.amazon.com/ecr/)
3. Configure data input channels
4. Configure hyper-prarameters
5. Define training metrics
6. Define training job and start training

Before we get started, let us initialize two python variables ```aws_region``` and ```s3_bucket``` that we will use throughout the notebook:

In [1]:
aws_region =  "ap-southeast-1"
s3_bucket  =  "smart-invoice"
! $(aws ecr get-login --no-include-email --region ap-southeast-1  --registry-ids 393782509758)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


## Stage COCO 2017 dataset in Amazon S3

We use [COCO 2017 dataset](http://cocodataset.org/#home) for training. We download COCO 2017 training and validation dataset to this notebook instance, extract the files from the dataset archives, and upload the extracted files to your Amazon [S3 bucket](https://docs.aws.amazon.com/en_pv/AmazonS3/latest/gsg/CreatingABucket.html) with the prefix ```mask-rcnn/sagemaker/input/train```. The ```prepare-s3-bucket.sh``` script executes this step.


In [14]:
!cat ./prepare-s3-bucket.sh

#!/bin/bash

set -e

if [ "$#" -ne 1 ]; then
    echo "usage: $0 <s3-bucket-name>"
    exit 1
fi

S3_BUCKET=$1
S3_PREFIX="mask-rcnn/sagemaker/input"

# Stage directory must be on EBS volume with 100 GB available space
STAGE_DIR=$HOME/SageMaker/coco-2017-$(date +"%Y-%m-%d-%H-%M-%S")


echo "Create stage directory: $STAGE_DIR"
mkdir -p $STAGE_DIR

wget -O $STAGE_DIR/train2017.zip http://images.cocodataset.org/zips/train2017.zip
echo "Extracting $STAGE_DIR/train2017.zip"
unzip -o $STAGE_DIR/train2017.zip  -d $STAGE_DIR | awk 'BEGIN {ORS="="} {if(NR%1000==0)print "="}'
echo "Done."
rm $STAGE_DIR/train2017.zip

wget -O $STAGE_DIR/val2017.zip http://images.cocodataset.org/zips/val2017.zip
echo "Extracting $STAGE_DIR/val2017.zip"
unzip -o $STAGE_DIR/val2017.zip -d $STAGE_DIR | awk 'BEGIN {ORS="="} {if(NR%1000==0)print "="}'
echo "Done."
rm $STAGE_DIR/val2017.zip

wget -O $STAGE_DIR/annotations_trainval2017.zip http://images.cocodataset.org/annotations/annotations_trainval2017.zip
unzip -o $ST

 Using your *Amazon S3 bucket* as argument, run the cell below. If you have already uploaded COCO 2017 dataset to your Amazon S3 bucket *in this AWS region*, you may skip this step. The expected time to execute this step is 20 minutes.

In [ ]:
%%time
!./prepare-s3-bucket.sh {s3_bucket}

## Build and push SageMaker training images

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to Amazon ECR service. If you created this notebook instance using the ```./stack-sm.sh``` script in this repository, the IAM Role attached to this notebook instance is already setup with full access to ECR service. 

Below, we have a choice of two different implementations:

1. [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) implementation supports a maximum per-GPU batch size of 1, and does not support mixed precision. It can be used with mainstream TensorFlow releases.

2. [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) is an optimized implementation that supports a maximum batch size of 4 and supports mixed precision. This implementation uses TensorFlow base version 1.13 augmented with custom TensorFlow ops. 

It is recommended that you build and push both SageMaker training images and use either image for training later.


### TensorPack Faster-RCNN/Mask-RCNN

Use ```./container/build_tools/build_and_push.sh``` script to build and push the TensorPack Faster-RCNN/Mask-RCNN  training image to Amazon ECR. 

In [ ]:
!cat ./container/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [15]:
%%time
! ./container/build_tools/build_and_push.sh {aws_region}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
^C
CPU times: user 92.1 ms, sys: 29.1 ms, total: 121 ms
Wall time: 5.34 s


Set ```tensorpack_image``` below to Amazon ECR URI of the image you pushed above.

In [16]:
tensorpack_image = "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorpack-sagemaker:tf1.13-tp26664c3"

### AWS Samples Mask R-CNN
Use ```./container-optimized/build_tools/build_and_push.sh``` script to build and push the AWS Samples Mask R-CNN training image to Amazon ECR.

In [2]:
!cat ./container-optimized/build_tools/build_and_push.sh

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source $DIR/set_env.sh

# set region
region=
if [ "$#" -eq 1 ]; then
    region=$1
else
    echo "usage: $0 <aws-region>"
    exit 1
fi
  

image=$IMAGE_NAME
tag=$IMAGE_TAG

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    aws ecr create-repository --region ${

Using your *AWS region* as argument, run the cell below.

In [3]:
%%time
! ./container-optimized/build_tools/build_and_push.sh {aws_region}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  20.48kB
Step 1/25 : FROM 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/invoice-extraction:tensorflow-base-1.13.1-gpu-py36-ubuntu-16.04
tensorflow-base-1.13.1-gpu-py36-ubuntu-16.04: Pulling from invoice-extraction

760c94fc: Pulling fs layer 
92f3c37b: Pulling fs layer 
e5e7f12e: Pulling fs layer 
74cc00ca: Pulling fs layer 
53113e13: Pulling fs layer 
2edc87fb: Pulling fs layer 
f57a58ef: Pulling fs layer 
aa495279: Pulling fs layer 
63b2d43d: Pulling fs layer 
d887aa10: Pulling fs layer 
aa4e079e: Pulling fs layer 
74cc00ca: Waiting fs layer 
aa4e079e: Waiting fs layer 
ddc7977b: Pulling fs layer 
16202287: Pulling fs layer 
ddc7977

92c2b346: Pull complete  959B/959B8MBBKDigest: sha256:9b8bdfe71e42e7755936d6d86b1eb73875dddbadbb26cf6f7fceb6576d7358e7


Status: Downloaded newer image for 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/invoice-extraction:tensorflow-base-1.13.1-gpu-py36-ubuntu-16.04
 ---> 678566651313
Step 2/25 : ENV HOROVOD_VERSION=0.18.1
 ---> Running in e1ac4bdc6304
Removing intermediate container e1ac4bdc6304
 ---> 18c79b9912d5
Step 3/25 : RUN pip install --upgrade pip
 ---> Running in 99acb05fccdc
Requirement already up-to-date: pip in /usr/local/lib/python3.6/site-packages (20.0.2)
Removing intermediate container 99acb05fccdc
 ---> 0760138a0b97
Step 4/25 : RUN pip uninstall -y tensorflow tensorboard tensorflow-estimator keras h5py horovod numpy
 ---> Running in 33a1526ffd64
Found existing installation: tensorflow 1.13.1
Uninstalling tensorflow-1.13.1:
  Successfully uninstalled tensorflow-1.13.1
Found existing installation: tensorflow-estimator 1.13.0
Uninstalling tensorflow-estimator-1.13.0:
  Successfully uninstalled tensorflow-estimator-1.13.0
Found existing installation: Keras 2.2.4
Uninstalling Keras-2.2.4:

  2200K .......... .......... .......... .......... ..........  1%  223K 1m49s
  2250K .......... .......... .......... .......... ..........  1% 22.4M 1m47s
  2300K .......... .......... .......... .......... ..........  1% 94.2M 1m44s
  2350K .......... .......... .......... .......... ..........  1% 99.7M 1m42s
  2400K .......... .......... .......... .......... ..........  1% 93.0M 1m40s
  2450K .......... .......... .......... .......... ..........  1% 97.1M 98s
  2500K .......... .......... .......... .......... ..........  1% 95.5M 96s
  2550K .......... .......... .......... .......... ..........  1%  101M 94s
  2600K .......... .......... .......... .......... ..........  2%  119M 93s
  2650K .......... .......... .......... .......... ..........  2%  125M 91s
  2700K .......... .......... .......... .......... ..........  2% 87.1M 89s
  2750K .......... .......... .......... .......... ..........  2% 94.2M 88s
  2800K .......... .......... .......... .......... ..........  2%

  6150K .......... .......... .......... .......... ..........  4%  227K 53s
  6200K .......... .......... .......... .......... ..........  4%  179M 52s
  6250K .......... .......... .......... .......... ..........  4%  175M 52s
  6300K .......... .......... .......... .......... ..........  4% 9.75M 52s
  6350K .......... .......... .......... .......... ..........  4% 11.7M 51s
  6400K .......... .......... .......... .......... ..........  4%  172M 51s
  6450K .......... .......... .......... .......... ..........  4%  149M 50s
  6500K .......... .......... .......... .......... ..........  5%  197M 50s
  6550K .......... .......... .......... .......... ..........  5%  177M 50s
  6600K .......... .......... .......... .......... ..........  5%  165M 49s
  6650K .......... .......... .......... .......... ..........  5%  142M 49s
  6700K .......... .......... .......... .......... ..........  5% 99.5M 48s
  6750K .......... .......... .......... .......... ..........  5% 48.0M 48s

...

127000K .......... .......... .......... .......... .......... 97% 63.6M 1s
127050K .......... .......... .......... .......... .......... 97% 16.1M 1s
127100K .......... .......... .......... .......... .......... 97% 79.8M 1s
127150K .......... .......... .......... .......... .......... 97% 63.4M 1s
127200K .......... .......... .......... .......... .......... 97%  153M 0s
127250K .......... .......... .......... .......... .......... 97% 55.0M 0s
127300K .......... .......... .......... .......... .......... 97% 80.1M 0s
127350K .......... .......... .......... .......... .......... 97%  113M 0s
127400K .......... .......... .......... .......... .......... 97% 72.6M 0s
127450K .......... .......... .......... .......... .......... 97% 86.3M 0s
127500K .......... .......... .......... .......... .......... 97% 65.0M 0s
127550K .......... .......... .......... .......... .......... 97% 68.1M 0s
127600K .......... .......... .......... .......... .......... 98%  173M 0s
127650K ....

ERROR: sagemaker-containers 2.8.6 has requirement scipy>=1.2.2, but you'll have scipy 1.2.1 which is incompatible.
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
Removing intermediate container c148e04960c7
 ---> 177d86b98b0f
Step 6/25 : RUN ldconfig /usr/local/cuda-10.0/targets/x86_64-linux/lib/stubs &&     HOROVOD_GPU_ALLREDUCE=NCCL HOROVOD_WITH_TENSORFLOW=1 pip install --no-cache-dir horovod==${HOROVOD_VERSION} &&     ldconfig
 ---> Running in 0ba10c6eb73e
  Created wheel for horovod: filename=horovod-0.18.1-cp36-cp36m-linux_x86_64.whl size=94085043 sha256=82de93501a720677cabcaa9749470faef1501d48bb5f6ecebc518e55cc32f6ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-bbnfa63i/wheels/97/63/83/a67efcfcc96182cf4f5f1ac081c9b6d514abd2bd42a9ba8834
Successfully built horovod
Removing intermediate container 0ba10c6eb73e
 ---> ec5f

  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.3.3
    Uninstalling s3transfer-0.3.3:
      Successfully uninstalled s3transfer-0.3.3
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1
    Uninstalling PyYAML-5.3.1:
      Successfully uninstalled PyYAML-5.3.1
Removing intermediate container 896aff06b902
 ---> 4bde9a079584
Step 11/25 : RUN pip install boto3
 ---> Running in 9c19ba28e430
ERROR: awscli 1.16.130 has requirement botocore==1.12.120, but you'll have botocore 1.15.43 which is incompatible.
ERROR: awscli 1.16.130 has requirement s3transfer<0.3.0,>=0.2.0, but you'll have s3transfer 0.3.3 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.12.120
    Uninstalling botocore-1.12.120:
      Successfully uninstalled botocore-1.12.120
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.2.1
    Uninstalling s3transfer-0.2.1:
      Successfully uninsta

  Running setup.py develop for tensorpack
Removing intermediate container 75da6b9a62a8
 ---> 0887ca90ca0b
Step 24/25 : COPY resources/train.py /opt/ml/code/train.py
 ---> 646685b4366f
Step 25/25 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 654676defdd4
Removing intermediate container 654676defdd4
 ---> 7c504364a538
Successfully built 7c504364a538
Successfully tagged mask-rcnn-tensorflow-sagemaker:latest
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker]

c6759c65: Preparing 
203ebcef: Preparing 
18be5120: Preparing 
0349e275: Preparing 
da03d291: Preparing 
d906ab6d: Preparing 
184db5ce: Preparing 
3bfc6512: Pr

356ba2a: Pushed   709.3MB/700.3MBPushing  300.6MB/700.3MBtf1.13-153442b: digest: sha256:c2ed0a23c6c34d0da82422a80ef8a9f7444d25e0c703320129d238289369860d size: 9340
Amazon ECR URI: 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker:tf1.13-153442b
CPU times: user 9.38 s, sys: 2.55 s, total: 11.9 s
Wall time: 7min 9s


 Set ```aws_samples_image``` below to Amazon ECR URI of the image you pushed above.

In [17]:
aws_samples_image = "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker:tf1.13-153442b"

## SageMaker Initialization 
We have staged the data and we have built and pushed the training docker image to Amazon ECR. Now we are ready to start using Amazon SageMaker.

In [28]:
%%time
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

SageMaker Execution Role:arn:aws:iam::393782509758:role/service-role/AmazonSageMaker-ExecutionRole-20191115T140457
AWS account:393782509758
AWS region:ap-southeast-1
CPU times: user 108 ms, sys: 2.12 ms, total: 110 ms
Wall time: 1.08 s


Next, we set ```training_image``` to the Amazon ECR image URI you saved in a previous step. 

In [29]:
training_image =  aws_samples_image# set to tensorpack_image or aws_samples_image 
print(f'Training image: {training_image}')

Training image: 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker:tf1.13-153442b


## Define SageMaker Data Channels
In this step, we define SageMaker *train* data channel. 

In [30]:
prefix = "mask-rcnn/sagemaker" #prefix in your S3 bucket

s3train = f's3://{s3_bucket}/{prefix}/input/invoice_dataset_400'


train = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/tfrecord', s3_data_type='S3Prefix')


data_channels = {'train': train}

Next, we define the model output location in S3 bucket.

In [31]:
s3_output_location = f's3://{s3_bucket}/{prefix}/output'

# Configure Hyper-parameters
Next, we define the hyper-parameters. 

Note, some hyper-parameters are different between the two implementations. The batch size per GPU in TensorPack Faster-RCNN/Mask-RCNN is fixed at 1, but is configurable in AWS Samples Mask-RCNN. The learning rate schedule is specified in units of steps in TensorPack Faster-RCNN/Mask-RCNN, but in epochs in AWS Samples Mask-RCNN.

The detault learning rate schedule values shown below correspond to training for a total of 24 epochs, at 120,000 images per epoch.

<table align='left'>
    <caption>TensorPack Faster-RCNN/Mask-RCNN  Hyper-parameters</caption>
    <tr>
    <th style="text-align:center">Hyper-parameter</th>
    <th style="text-align:center">Description</th>
    <th style="text-align:center">Default</th>
    </tr>
    <tr>
        <td style="text-align:center">mode_fpn</td>
        <td style="text-align:left">Flag to indicate use of Feature Pyramid Network (FPN) in the Mask R-CNN model backbone</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">mode_mask</td>
        <td style="text-align:left">A value of "False" means Faster-RCNN model, "True" means Mask R-CNN moodel</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">eval_period</td>
        <td style="text-align:left">Number of epochs period for evaluation during training</td>
        <td style="text-align:center">1</td>
    </tr>
    <tr>
        <td style="text-align:center">lr_schedule</td>
        <td style="text-align:left">Learning rate schedule in training steps</td>
        <td style="text-align:center">'[240000, 320000, 360000]'</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_norm</td>
        <td style="text-align:left">Batch normalization option ('FreezeBN', 'SyncBN', 'GN', 'None') </td>
        <td style="text-align:center">'FreezeBN'</td>
    </tr>
    <tr>
        <td style="text-align:center">images_per_epoch</td>
        <td style="text-align:left">Images per epoch </td>
        <td style="text-align:center">120000</td>
    </tr>
    <tr>
        <td style="text-align:center">data_train</td>
        <td style="text-align:left">Training data under data directory</td>
        <td style="text-align:center">'coco_train2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">data_val</td>
        <td style="text-align:left">Validation data under data directory</td>
        <td style="text-align:center">'coco_val2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">resnet_arch</td>
        <td style="text-align:left">Must be 'resnet50' or 'resnet101'</td>
        <td style="text-align:center">'resnet50'</td>
    </tr>
    <tr>
        <td style="text-align:center">backbone_weights</td>
        <td style="text-align:left">Pre-trained model weights</td>
        <td style="text-align:center">'ImageNet-R50-AlignPadding.npz'</td>
    </tr>
</table>

    
<table align='left'>
    <caption>AWS Samples Mask-RCNN  Hyper-parameters</caption>
    <tr>
    <th style="text-align:center">Hyper-parameter</th>
    <th style="text-align:center">Description</th>
    <th style="text-align:center">Default</th>
    </tr>
    <tr>
        <td style="text-align:center">mode_fpn</td>
        <td style="text-align:left">Flag to indicate use of Feature Pyramid Network (FPN) in the Mask R-CNN model backbone</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">mode_mask</td>
        <td style="text-align:left">A value of "False" means Faster-RCNN model, "True" means Mask R-CNN moodel</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">eval_period</td>
        <td style="text-align:left">Number of epochs period for evaluation during training</td>
        <td style="text-align:center">1</td>
    </tr>
    <tr>
        <td style="text-align:center">lr_epoch_schedule</td>
        <td style="text-align:left">Learning rate schedule in epochs</td>
        <td style="text-align:center">'[(16, 0.1), (20, 0.01), (24, None)]'</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_size_per_gpu</td>
        <td style="text-align:left">Batch size per gpu ( Minimum 1, Maximum 4)</td>
        <td style="text-align:center">4</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_norm</td>
        <td style="text-align:left">Batch normalization option ('FreezeBN', 'SyncBN', 'GN', 'None') </td>
        <td style="text-align:center">'FreezeBN'</td>
    </tr>
    <tr>
        <td style="text-align:center">images_per_epoch</td>
        <td style="text-align:left">Images per epoch </td>
        <td style="text-align:center">120000</td>
    </tr>
    <tr>
        <td style="text-align:center">data_train</td>
        <td style="text-align:left">Training data under data directory</td>
        <td style="text-align:center">'train2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">data_val</td>
        <td style="text-align:left">Validation data under data directory</td>
        <td style="text-align:center">'val2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">resnet_arch</td>
        <td style="text-align:left">Must be 'resnet50' or 'resnet101'</td>
        <td style="text-align:center">'resnet50'</td>
    </tr>
    <tr>
        <td style="text-align:center">backbone_weights</td>
        <td style="text-align:left">Pre-trained model weights</td>
        <td style="text-align:center">'ImageNet-R50-AlignPadding.npz'</td>
    </tr>
</table>

In [32]:
hyperparameters = {
                    "mode_fpn": "True",
                    "mode_mask": "True",
                    "eval_period": 10,
                    "batch_norm": "FreezeBN",
                    "batch_size_per_gpu": 1,
                    "images_per_epoch": 100,
                    "lr_epoch_schedule": '[(160, 0.1), (200, 0.01), (240, None)]'
                  }

## Define Training Metrics
Next, we define the regular expressions that SageMaker uses to extract algorithm metrics from training logs and send them to [AWS CloudWatch metrics](https://docs.aws.amazon.com/en_pv/AmazonCloudWatch/latest/monitoring/working_with_metrics.html). These algorithm metrics are visualized in SageMaker console.

In [33]:
metric_definitions=[
             {
                "Name": "fastrcnn_losses/box_loss",
                "Regex": ".*fastrcnn_losses/box_loss:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_loss",
                "Regex": ".*fastrcnn_losses/label_loss:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/accuracy",
                "Regex": ".*fastrcnn_losses/label_metrics/accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/false_negative",
                "Regex": ".*fastrcnn_losses/label_metrics/false_negative:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/fg_accuracy",
                "Regex": ".*fastrcnn_losses/label_metrics/fg_accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/num_fg_label",
                "Regex": ".*fastrcnn_losses/num_fg_label:\\s*(\\S+).*"
            },
             {
                "Name": "maskrcnn_loss/accuracy",
                "Regex": ".*maskrcnn_loss/accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/fg_pixel_ratio",
                "Regex": ".*maskrcnn_loss/fg_pixel_ratio:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/maskrcnn_loss",
                "Regex": ".*maskrcnn_loss/maskrcnn_loss:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/pos_accuracy",
                "Regex": ".*maskrcnn_loss/pos_accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.5",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.5:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.5:0.95",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.5:0\\.95:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.75",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.75:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/large",
                "Regex": ".*mAP\\(bbox\\)/large:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/medium",
                "Regex": ".*mAP\\(bbox\\)/medium:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/small",
                "Regex": ".*mAP\\(bbox\\)/small:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.5",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.5:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.5:0.95",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.5:0\\.95:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.75",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.75:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/large",
                "Regex": ".*mAP\\(segm\\)/large:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/medium",
                "Regex": ".*mAP\\(segm\\)/medium:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/small",
                "Regex": ".*mAP\\(segm\\)/small:\\s*(\\S+).*"
            }  
            
    ]

## Define SageMaker Training Job

Next, we use SageMaker [Estimator](https://sagemaker.readthedocs.io/en/stable/estimators.html) API to define a SageMaker Training Job. 

We recommned using 32 GPUs, so we set ```train_instance_count=4``` and ```train_instance_type='ml.p3.16xlarge'```, because there are 8 Tesla V100 GPUs per ```ml.p3.16xlarge``` instance. We recommend using 100 GB [Amazon EBS](https://aws.amazon.com/ebs/) storage volume with each training instance, so we set ```train_volume_size = 100```. We want to replicate training data to each training instance, so we set ```input_mode= 'File'```.

We run the training job in your private VPC, so we need to set the ```subnets``` and ```security_group_ids``` prior to running the cell below. You may specify multiple subnet ids in the ```subnets``` list. The subnets included in the ```sunbets``` list must be part of the output of  ```./stack-sm.sh``` CloudFormation stack script used to create this notebook instance. Specify only one security group id in ```security_group_ids``` list. The security group id must be part of the output of  ```./stack-sm.sh``` script.

For ```train_instance_type``` below, you have the option to use ```ml.p3.16xlarge``` with 16 GB per-GPU memory and 25 Gbs network interconnectivity, or ```ml.p3dn.24xlarge``` with 32 GB per-GPU memory and 100 Gbs network interconnectivity. The ```ml.p3dn.24xlarge``` instance type offers significantly better performance than ```ml.p3.16xlarge``` for Mask R-CNN distributed TensorFlow training.

In [34]:
# security_group_ids =  # ['sg-xxxxxxxx']
# subnets =      # [ 'subnet-xxxxxxx']
sagemaker_session = sagemaker.session.Session(boto_session=session)
mask_rcnn_estimator = Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.2xlarge',
                                         #train_instance_type='local_gpu',
                                         train_volume_size = 100,
                                         train_max_run = 10000,
                                         train_max_wait = 10000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sagemaker_session, 
                                         hyperparameters = hyperparameters,
                                         metric_definitions = metric_definitions,
                                         train_use_spot_instances=True,
                                         base_job_name="mask-rcnn-s3")

Finally, we launch the SageMaker training job. 

The estimated time for downloading data to all the training instances is 20 minutes. The time to complete the training depends on type and number of training instances, and the training image used for training.

In [35]:
mask_rcnn_estimator.fit(inputs=data_channels, logs="All")

2020-04-05 15:14:52 Starting - Starting the training job......
2020-04-05 15:15:23 Starting - Launching requested ML instances......
2020-04-05 15:16:24 Starting - Preparing the instances for training.........
2020-04-05 15:18:09 Downloading - Downloading input data
2020-04-05 15:18:09 Training - Downloading the training image............
2020-04-05 15:20:14 Training - Training image download completed. Training in progress../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8",

[1,0]<stdout>:#033[0405 15:20:29 @registry.py:135]#033 group3/block1/conv2 output: [None, 512, None, None]
[1,0]<stdout>:#033[0405 15:20:29 @registry.py:127]#033 group3/block1/conv3 input: [None, 512, None, None]
[1,0]<stdout>:#033[0405 15:20:29 @batch_norm.py:166]#033 #033#033WRN#033 [BatchNorm] Using moving_mean/moving_variance in training.
[1,0]<stdout>:#033[0405 15:20:29 @registry.py:135]#033 group3/block1/conv3 output: [None, 2048, None, None]
[1,0]<stdout>:#033[0405 15:20:29 @registry.py:127]#033 group3/block2/conv1 input: [None, 2048, None, None]
[1,0]<stdout>:#033[0405 15:20:29 @batch_norm.py:166]#033 #033#033WRN#033 [BatchNorm] Using moving_mean/moving_variance in training.
[1,0]<stdout>:#033[0405 15:20:29 @registry.py:135]#033 group3/block2/conv1 output: [None, 512, None, None]
[1,0]<stdout>:#033[0405 15:20:29 @registry.py:127]#033 group3/block2/conv2 input: [None, 512, None, None]
[1,0]<stdout>:#033[0405 15:20:29 @batch_norm.py:166]#033 #033#033WRN#033 [BatchNorm] Using movi

[1,0]<stdout>:#033[0405 15:20:45 @collection.py:153]#033 Size of these collections were changed in tower-pred-0: (tf.GraphKeys.MODEL_VARIABLES: 183->238)
[1,0]<stdout>:loading annotations into memory...
[1,0]<stdout>:Done (t=0.00s)
[1,0]<stdout>:creating index...
[1,0]<stdout>:index created!
[1,0]<stdout>:#033[0405 15:20:45 @dataset.py:50]#033 Instances loaded from /opt/ml/input/data/train/annotations/instances_val2017.json.
[1,0]<stdout>:#033[0405 15:20:45 @timer.py:50]#033 Load Groundtruth Boxes for val2017 finished, time:0.0096sec.
[1,0]<stdout>:#033[0405 15:20:45 @summary.py:48]#033 [MovingAverageSummary] 27 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[1,0]<stdout>:#033[0405 15:20:45 @summary.py:95]#033 Summarizing collection 'summaries' of size 30.
[1,0]<stdout>:#033[0405 15:20:48 @base.py:231]#033 Creating the session ...
[1,0]<stdout>:#033[0405 15:20:53 @base.py:237]#033 Initializing the session ...
[1,0]<stdout>:#033[0405 15:20:53 @sessinit.py:

[1,0]<stdout>:#033[0405 15:21:03 @varmanip.py:104]#033 #033#033WRN#033 Variable group0/block0/conv2/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0405 15:21:03 @varmanip.py:104]#033 #033#033WRN#033 Variable conv0/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0405 15:21:04 @varmanip.py:104]#033 #033#033WRN#033 Variable group0/block0/conv1/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0405 15:21:05 @varmanip.py:104]#033 #033#033WRN#033 Variable group0/block2/conv1/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0405 15:21:08 @varmanip.py:104]#033 #033#033WRN#033 Variable group0/block0/conv3/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0405 

[1,0]<stdout>:#033[0405 15:22:20 @base.py:286]#033 Epoch 3 (global_step 300) finished, time:9.65 seconds.
[1,0]<stdout>:#033[0405 15:22:20 @trainers.py:419]#033 Running horovod broadcast ...
[1,0]<stdout>:#033[0405 15:22:20 @misc.py:111]#033 Estimated Time Left: 38 minutes 55 seconds
[1,0]<stdout>:#033[0405 15:22:20 @performance.py:179]#033 [ThroughputTracker] Over last epoch, MeanEpochThroughput: 10.37
[1,0]<stdout>:#033[0405 15:22:20 @performance.py:180]#033 [ThroughputTracker] Over last epoch, EpochWallClockDuration: 9.65
[1,0]<stdout>:#033[0405 15:22:20 @performance.py:181]#033 [ThroughputTracker] Over last epoch, CallbackOverheadDuration: 0.00
[1,0]<stdout>:#033[0405 15:22:20 @monitor.py:469]#033 PeakMemory(MB)/gpu:0: 4270.1
[1,0]<stdout>:#033[0405 15:22:20 @monitor.py:469]#033 QueueInput/queue_size: 50
[1,0]<stdout>:#033[0405 15:22:20 @monitor.py:469]#033 Throughput/CallbackOverheadDuration: 4.4823e-05
[1,0]<stdout>:#033[0405 15:22:20 @monitor.py:469]#033 Throughput/EpochWallCloc

...

[1,0]<stdout>:#033[0405 15:59:50 @base.py:286]#033 Epoch 233 (global_step 23300) finished, time:9.1 seconds.
[1,0]<stdout>:#033[0405 15:59:50 @trainers.py:419]#033 Running horovod broadcast ...
[1,0]<stdout>:#033[0405 15:59:50 @misc.py:111]#033 Estimated Time Left: 1 minute 4 seconds
[1,0]<stdout>:#033[0405 15:59:50 @performance.py:179]#033 [ThroughputTracker] Over last epoch, MeanEpochThroughput: 10.99
[1,0]<stdout>:#033[0405 15:59:50 @performance.py:180]#033 [ThroughputTracker] Over last epoch, EpochWallClockDuration: 9.10
[1,0]<stdout>:#033[0405 15:59:50 @performance.py:181]#033 [ThroughputTracker] Over last epoch, CallbackOverheadDuration: 0.00
[1,0]<stdout>:#033[0405 15:59:50 @monitor.py:469]#033 PeakMemory(MB)/gpu:0: 5050.4
[1,0]<stdout>:#033[0405 15:59:50 @monitor.py:469]#033 QueueInput/queue_size: 50
[1,0]<stdout>:#033[0405 15:59:50 @monitor.py:469]#033 Throughput/CallbackOverheadDuration: 3.8147e-05
[1,0]<stdout>:#033[0405 15:59:50 @monitor.py:469]#033 Throughput/EpochWallCloc

[1,0]<stdout>:#033[0405 16:00:08 @base.py:286]#033 Epoch 235 (global_step 23500) finished, time:9.1 seconds.
[1,0]<stdout>:#033[0405 16:00:08 @trainers.py:419]#033 Running horovod broadcast ...
[1,0]<stdout>:#033[0405 16:00:08 @misc.py:111]#033 Estimated Time Left: 46 seconds
[1,0]<stdout>:#033[0405 16:00:08 @performance.py:179]#033 [ThroughputTracker] Over last epoch, MeanEpochThroughput: 10.99
[1,0]<stdout>:#033[0405 16:00:08 @performance.py:180]#033 [ThroughputTracker] Over last epoch, EpochWallClockDuration: 9.10
[1,0]<stdout>:#033[0405 16:00:08 @performance.py:181]#033 [ThroughputTracker] Over last epoch, CallbackOverheadDuration: 0.00
[1,0]<stdout>:#033[0405 16:00:08 @monitor.py:469]#033 PeakMemory(MB)/gpu:0: 5050.4
[1,0]<stdout>:#033[0405 16:00:08 @monitor.py:469]#033 QueueInput/queue_size: 50
[1,0]<stdout>:#033[0405 16:00:08 @monitor.py:469]#033 Throughput/CallbackOverheadDuration: 3.6955e-05
[1,0]<stdout>:#033[0405 16:00:08 @monitor.py:469]#033 Throughput/EpochWallClockDuratio

[1,0]<stdout>:#033[0405 16:00:26 @base.py:286]#033 Epoch 237 (global_step 23700) finished, time:9.05 seconds.
[1,0]<stdout>:#033[0405 16:00:26 @trainers.py:419]#033 Running horovod broadcast ...
[1,0]<stdout>:#033[0405 16:00:26 @misc.py:111]#033 Estimated Time Left: 27.5 seconds
[1,0]<stdout>:#033[0405 16:00:26 @performance.py:179]#033 [ThroughputTracker] Over last epoch, MeanEpochThroughput: 11.05
[1,0]<stdout>:#033[0405 16:00:26 @performance.py:180]#033 [ThroughputTracker] Over last epoch, EpochWallClockDuration: 9.05
[1,0]<stdout>:#033[0405 16:00:26 @performance.py:181]#033 [ThroughputTracker] Over last epoch, CallbackOverheadDuration: 0.00
[1,0]<stdout>:#033[0405 16:00:26 @monitor.py:469]#033 PeakMemory(MB)/gpu:0: 5050.4
[1,0]<stdout>:#033[0405 16:00:26 @monitor.py:469]#033 QueueInput/queue_size: 50
[1,0]<stdout>:#033[0405 16:00:26 @monitor.py:469]#033 Throughput/CallbackOverheadDuration: 4.4107e-05
[1,0]<stdout>:#033[0405 16:00:26 @monitor.py:469]#033 Throughput/EpochWallClockDura

[1,0]<stdout>:#033[0405 16:00:45 @base.py:286]#033 Epoch 239 (global_step 23900) finished, time:9.07 seconds.
[1,0]<stdout>:#033[0405 16:00:45 @trainers.py:419]#033 Running horovod broadcast ...
[1,0]<stdout>:#033[0405 16:00:45 @misc.py:111]#033 Estimated Time Left: 9.16 seconds
[1,0]<stdout>:#033[0405 16:00:45 @performance.py:179]#033 [ThroughputTracker] Over last epoch, MeanEpochThroughput: 11.03
[1,0]<stdout>:#033[0405 16:00:45 @performance.py:180]#033 [ThroughputTracker] Over last epoch, EpochWallClockDuration: 9.07
[1,0]<stdout>:#033[0405 16:00:45 @performance.py:181]#033 [ThroughputTracker] Over last epoch, CallbackOverheadDuration: 0.00
[1,0]<stdout>:#033[0405 16:00:45 @monitor.py:469]#033 PeakMemory(MB)/gpu:0: 5050.4
[1,0]<stdout>:#033[0405 16:00:45 @monitor.py:469]#033 QueueInput/queue_size: 50
[1,0]<stdout>:#033[0405 16:00:45 @monitor.py:469]#033 Throughput/CallbackOverheadDuration: 6.1989e-05
[1,0]<stdout>:#033[0405 16:00:45 @monitor.py:469]#033 Throughput/EpochWallClockDura


2020-04-05 16:08:25 Completed - Training job completed
Training seconds: 3027
Billable seconds: 772
Managed Spot Training savings: 74.5%
